# Importing Basic Package

In [23]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Load the data

In [25]:
customers = pd.read_csv('Customers.csv')
transactions = pd.read_csv('Transactions.csv')

# Merge transactions with customers to get customer details

In [27]:
merged_data = pd.merge(transactions, customers, on='CustomerID')

# Feature Engineering

In [29]:
customer_features = merged_data.groupby('CustomerID').agg({
    'TotalValue': ['sum', 'mean'],
    'Quantity': ['sum', 'mean'],
    'ProductID': 'nunique'
}).reset_index()

customer_features.columns = ['CustomerID', 'TotalSpending', 'AvgTransactionValue', 'TotalQuantity', 'AvgQuantity', 'UniqueProducts']

# Merge with customer profile information

In [31]:
customer_features = pd.merge(customer_features, customers, on='CustomerID')

# Encode categorical variables (Region)

In [33]:
customer_features = pd.get_dummies(customer_features, columns=['Region'])

# Normalize the features

In [35]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_features.drop(columns=['CustomerID', 'CustomerName', 'SignupDate']))

# Calculate similarity matrix

In [39]:
similarity_matrix = cosine_similarity(scaled_features)

# Create a DataFrame for similarity scores

In [41]:
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])

# Function to get top 3 similar customers

In [43]:
def get_top_similar_customers(customer_id, similarity_df, top_n=3):
    similar_customers = similarity_df[customer_id].sort_values(ascending=False).iloc[1:top_n+1]
    return similar_customers.index.tolist(), similar_customers.values.tolist()

# Generate recommendations for the first 20 customers

In [45]:
lookalike_map = {}
for customer_id in customer_features['CustomerID'][:20]:
    similar_customers, scores = get_top_similar_customers(customer_id, similarity_df)
    lookalike_map[customer_id] = list(zip(similar_customers, scores))

# Save to CSV

In [47]:
lookalike_df = pd.DataFrame(lookalike_map.items(), columns=['CustomerID', 'Lookalikes'])
lookalike_df.to_csv('Rayvan_Kumar_V_Lookalike.csv', index=False)